In [ ]:
# !pip3 install spotify
# !pip3 install image
# !pip3 install Send2Trash
# !pip3 install matplotlib

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import requests
from PIL import Image
from io import BytesIO
import math
import os
from datetime import datetime

In [ ]:
# Spotify API credentials, the guide of getting it provided in README.md
client_id = os.environ['SPOTIFY_CLIENT_ID']
client_secret = os.environ['SPOTIFY_CLIENT_SECRET']
redirect_uri = 'http://localhost:8888/callback'

# Authenticate with Spotify
scope = 'user-library-read user-read-currently-playing user-read-playback-state user-read-recently-played'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope=scope))

In [ ]:
# Fetch all saved albums
results = sp.current_user_saved_albums(limit=50)
albums = results['items']

print('making...')

while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

# Extract album cover URLs
cover_urls = [album['album']['images'][0]['url'] for album in albums]

# Download album cover images
images = []
for url in cover_urls:
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    images.append(img)

In [ ]:
# Determine the size of the collage
num_images = len(images)
cols = int(math.sqrt(num_images))
rows = math.ceil(num_images / cols)
thumbnail_width = min(img.size[0] for img in images)
thumbnail_height = min(img.size[1] for img in images)

# Create a blank canvas for the collage
collage_width = cols * thumbnail_width
collage_height = rows * thumbnail_height
collage = Image.new('RGB', (collage_width, collage_height))

In [ ]:
# Paste images into the collage
for i, img in enumerate(images):
    img.thumbnail((thumbnail_width, thumbnail_height))
    x = (i % cols) * thumbnail_width
    y = (i // cols) * thumbnail_height
    collage.paste(img, (x, y))

# Save the collage
collage_dir = "./collages/"
if not os.path.exists(collage_dir):
    os.makedirs(collage_dir)

current_time = str(int(datetime.now().strftime('%Y%m%d%H%M%S')))

file_name = current_time + "_collage.jpeg"

file_path = collage_dir + file_name

collage.save(file_path)
print(f"Collage saved as {file_name} in collage folder")